In [1]:
import os
os.chdir(r'C:\Users\82107\Desktop\Study\ML learning')
print('current directory:', os.getcwd())

current directory: C:\Users\82107\Desktop\Study\ML learning


In [2]:
# 행렬을 다루기 위한 모듈을 불러온다.
import numpy as np
# 데이터를 읽어오기 위한 pandas 모듈을 불러온다.
import pandas as pd 

# csv 파일 형식으로 되어있는 advertising 데이터를 불러온다.
advertising = pd.read_csv('Advertising.csv', encoding='cp949', index_col=0)

# 데이터의 개수
len(advertising)

200

In [3]:
# train, validation, test set의 분할
train = advertising[:100]
train.head()

,TV,radio,newspaper,sales
1,230.1,37.8,69.2,22.1
2,44.5,39.3,45.1,10.4
3,17.2,45.9,69.3,9.3
4,151.5,41.3,58.5,18.5
5,180.8,10.8,58.4,12.9


In [4]:
val = advertising[50:150]
val.head()

,TV,radio,newspaper,sales
51,199.8,3.1,34.6,11.4
52,100.4,9.6,3.6,10.7
53,216.4,41.7,39.6,22.6
54,182.6,46.2,58.7,21.2
55,262.7,28.8,15.9,20.2


In [5]:
test = advertising[150:]
test.head()

#데이터셋의 크기에 맞춰서 랜덤하게 불러오면 됨

,TV,radio,newspaper,sales
151,280.7,13.9,37.0,16.1
152,121.0,8.4,48.7,11.6
153,197.6,23.3,14.2,16.6
154,171.3,39.7,37.7,19.0
155,187.8,21.1,9.5,15.6


In [6]:
# Bestsubset Selection에서 사용될 predictor들의 조합을 생성 (모든 변수의 조합들을 고려한 모형 전체)
import itertools
from copy import deepcopy
predictors = ['TV', 'radio', 'newspaper']
bestsubset = deepcopy(predictors)
for i in range(2, len(predictors)+1):
    bestsubset.extend([list(x) for x in list(itertools.combinations(predictors, i))])
bestsubset

['TV',
 'radio',
 'newspaper',
 ['TV', 'radio'],
 ['TV', 'newspaper'],
 ['radio', 'newspaper'],
 ['TV', 'radio', 'newspaper']]

In [8]:
import statsmodels.api as sm
# 7개의 모형의 subset들에 대해서 모형을 적합하고
# validation set에 대한 MSE를 계산
val_mse = []
for i in range(len(bestsubset)):
    simple_reg = sm.OLS(train['sales'], train[bestsubset[i]]).fit() #sm.OLS가 linear regression 모델을 적합하는 것임(데이터 적합)
    simple_reg_pred = np.array(simple_reg.predict(val[bestsubset[i]]))
    mse = (((simple_reg_pred - np.array(val['sales'])) ** 2).sum()) / len(simple_reg_pred) #MSE를 구하는 코드
    val_mse.append(mse)
val_mse # 이 결과가 validation error임 (여기서 가장 작은 값을 선택하면 됨)

[22.332072458685033,
 40.98359486189848,
 72.2655317909861,
 4.407420989798702,
 16.02429464820829,
 36.09094859100722,
 4.234901458264232]

In [9]:
# 가장 작은 validation MSE를 가지는 subset을 선택
np.argsort(val_mse)[0]
min_mse_subset = bestsubset[np.argsort(val_mse)[0]]
min_mse_subset 

['TV', 'radio', 'newspaper']

In [11]:
# 앞에서 고른 bestsubset를 이용하여 test 데이터에 대한 MSE를 계산
best_subset_reg = sm.OLS(train['sales'], train[min_mse_subset]).fit()
best_subset_reg_pred = np.array(best_subset_reg.predict(test[min_mse_subset]))
test_mse = (((best_subset_reg_pred - np.array(test['sales'])) ** 2).sum()) / len(best_subset_reg_pred)
test_mse 

3.6851575325875685

In [14]:
advertising = pd.read_csv('Advertising.csv', encoding='cp949', index_col=0) 

# 5-fold cross validation
k = 5
# 각 validation set의 크기
val_size = int(len(advertising) / k)
val_size

40

In [15]:
predictors = ['TV', 'radio', 'newspaper']
# 1개의 validation set을 분할
# -> 관측치들의 번호를 분할
i = 0
idx = np.arange(len(advertising))
cv_val_idx = np.arange(i*val_size, (i+1)*val_size)
cv_val_idx 

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39])

In [16]:
cv_train_idx = np.array([x for x in idx if x not in cv_val_idx])
cv_train_idx 

array([ 40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104,
       105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117,
       118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130,
       131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143,
       144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156,
       157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169,
       170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182,
       183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195,
       196, 197, 198, 199])

In [17]:
# 1개의 cross-validation set을 이용하여 MSE를 계산
reg = sm.OLS(advertising['sales'].iloc[cv_train_idx], # train set을 이용
             advertising[predictors].iloc[cv_train_idx]).fit()
cv_pred = np.array(reg.predict(advertising[predictors].iloc[cv_val_idx]))
mse = (((cv_pred - np.array(advertising['sales'].iloc[cv_val_idx])) ** 2).sum()) / len(cv_pred)
mse 

4.588441056058844

In [19]:
# 모든 5개의 cross-validation set에 대해서 MSE를 계산
cv_mse = []
for i in range(k):
    idx = np.arange(len(advertising))
    cv_val_idx = np.arange(i*val_size, (i+1)*val_size)
    cv_train_idx = np.array([x for x in idx if x not in cv_val_idx])
    reg = sm.OLS(advertising['sales'].iloc[cv_train_idx], 
                 advertising[predictors].iloc[cv_train_idx]).fit()
    cv_pred = np.array(reg.predict(advertising[predictors].iloc[cv_val_idx]))
    mse = (((cv_pred - np.array(advertising['sales'].iloc[cv_val_idx])) ** 2).sum()) / len(cv_pred)
    cv_mse.append(mse)

# 각 cross-validation set들의 MSE 결과
cv_mse

[4.588441056058844,
 3.819649621611199,
 3.5214234286826027,
 4.780892181762026,
 4.135290964081369]

In [20]:
# 5개의 cross-validation MSE의 평균
np.sum(cv_mse) / k

4.169139450439208